In [1]:
import pandas as pd

data = pd.read_excel(r'C:\Users\qyypy\Desktop\机器学习综合实践\Data\dataSet1\train.xlsx')
data.head()

,Patient Code,Group,Glucose,Urea,Creatinine,Sodium,Potassium,TB,DB,ALT,...,WBC,Platelet,CRP,PCT,IL-6,PT,D-Dimer,Troponin,CPK-MB,LDH
0,C 144,Control,72.0,35.0,1.1,138.0,4.4,0.6,0.1,57.0,...,3900.0,259000.0,85.6,0.9,NaN,22.0,0.3,Negative,0.6,10080.0
1,C 90,Control,80.0,16.0,1.2,130.0,4.5,0.3,0.5,46.0,...,6600.0,242000.0,10.3,0.1,NaN,13.0,0.1,Negative,46.0,9800.0
2,C 498,Control,83.0,44.0,0.1,138.0,4.7,0.5,0.3,35.0,...,7200.0,118000.0,0.0,0.4,NaN,13.0,0.2,Negative,30.0,8060.0
3,C 270,Control,150.0,33.0,0.9,139.0,3.7,0.9,0.4,21.0,...,4800.0,155000.0,0.0,0.1,NaN,13.0,1.0,Negative,16.0,5010.0
4,C 217,Control,72.0,25.0,1.0,139.0,4.1,1.0,0.2,62.0,...,8300.0,160000.0,0.0,0.2,NaN,13.0,0.2,Negative,22.0,4370.0


In [ ]:
data.shape

In [ ]:
data.head().info()

In [2]:
# 删除缺失率过高的变量
def filter_nan(data, narate = 0.2):
    n_samples = data.shape[0]
    list_nan_cols = []
    
    for col in data.columns:
        if data[col].isna().sum() / n_samples >= (narate):
            list_nan_cols.append(col)
    
    print(f'缺失量在{narate * 100}%以上的变量有:{list_nan_cols}')

    return list_nan_cols

In [3]:
list_nullfactor_todrop = filter_nan(data, narate=0.2)

data_select = data.drop(list_nullfactor_todrop, axis = 1).copy()
data_select.head()

缺失量在20.0%以上的变量有:['Total Protein', 'Albubin', 'Ferritin', 'PCT', 'IL-6', 'PT', 'Troponin', 'CPK-MB', 'LDH']


,Patient Code,Group,Glucose,Urea,Creatinine,Sodium,Potassium,TB,DB,ALT,AST,ALP,Hemoglobin,WBC,Platelet,CRP,D-Dimer
0,C 144,Control,72.0,35.0,1.1,138.0,4.4,0.6,0.1,57.0,99.0,86.0,9.9,3900.0,259000.0,85.6,0.3
1,C 90,Control,80.0,16.0,1.2,130.0,4.5,0.3,0.5,46.0,59.0,221.0,13.3,6600.0,242000.0,10.3,0.1
2,C 498,Control,83.0,44.0,0.1,138.0,4.7,0.5,0.3,35.0,30.0,101.0,10.5,7200.0,118000.0,0.0,0.2
3,C 270,Control,150.0,33.0,0.9,139.0,3.7,0.9,0.4,21.0,39.0,96.0,14.6,4800.0,155000.0,0.0,1.0
4,C 217,Control,72.0,25.0,1.0,139.0,4.1,1.0,0.2,62.0,35.0,74.0,13.2,8300.0,160000.0,0.0,0.2


In [4]:
# KNN处理缺失值
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

numerical_cols = data_select.select_dtypes(include=['float64', 'int64']).columns

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_select[numerical_cols])

knn_imputer = KNNImputer(n_neighbors=8)
imputed_data = knn_imputer.fit_transform(scaled_data)

imputed_data = scaler.inverse_transform(imputed_data)


data_select[numerical_cols] = imputed_data



In [5]:
# Group如果是新冠患者改成1,否则为0,发现病人编号没用，删去
data_select['Group '] = data_select['Group '].apply(lambda x: 0 if x == 'Control' else 1)
data_select = data_select.drop(columns = 'Patient Code')
data_select.to_csv("PreprocessedData.csv")
data_select

,Group,Glucose,Urea,Creatinine,Sodium,Potassium,TB,DB,ALT,AST,ALP,Hemoglobin,WBC,Platelet,CRP,D-Dimer
0,0,72.0,35.0,1.1,138.0,4.4000,0.6000,0.1000,57.0,99.00,86.000,9.9,3900.0,259000.0,85.6000,0.3000
1,0,80.0,16.0,1.2,130.0,4.5000,0.3000,0.5000,46.0,59.00,221.000,13.3,6600.0,242000.0,10.3000,0.1000
2,0,83.0,44.0,0.1,138.0,4.7000,0.5000,0.3000,35.0,30.00,101.000,10.5,7200.0,118000.0,0.0000,0.2000
3,0,150.0,33.0,0.9,139.0,3.7000,0.9000,0.4000,21.0,39.00,96.000,14.6,4800.0,155000.0,0.0000,1.0000
4,0,72.0,25.0,1.0,139.0,4.1000,1.0000,0.2000,62.0,35.00,74.000,13.2,8300.0,160000.0,0.0000,0.2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1819,1,169.0,86.0,0.9,141.0,3.5000,1.0000,0.3000,37.0,41.00,70.000,14.6,8400.0,254000.0,104.6250,1.0375
1820,1,141.0,45.0,1.1,146.0,3.2000,0.9125,0.2875,58.5,49.75,275.875,13.0,26600.0,253000.0,56.4750,1.0250
1821,1,127.0,33.0,0.8,140.0,3.8125,0.8000,0.2000,70.0,86.00,142.000,12.9,4500.0,250000.0,54.1375,0.4125
1822,1,164.0,202.0,6.2,131.0,4.8000,0.8000,0.2000,23.0,26.00,78.000,6.9,5000.0,250000.0,86.9125,3.3375


In [6]:
from sklearn.model_selection import train_test_split
# 选择特征和标签
X = data_select.drop(columns=['Group '])
y = data_select['Group ']

# 将数据集划分为训练集和验证集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=20)

X_train

,Glucose,Urea,Creatinine,Sodium,Potassium,TB,DB,ALT,AST,ALP,Hemoglobin,WBC,Platelet,CRP,D-Dimer
540,177.000,89.0,1.0,140.0,4.0,1.000,0.3000,45.000,37.000,107.000,8.6000,9700.0,74000.0,80.00,1.80
710,188.375,62.0,1.6,131.0,3.7,0.925,0.3375,57.625,43.875,113.125,13.2625,10150.0,238375.0,69.25,1.55
428,176.000,49.0,1.2,145.0,3.0,0.900,0.3000,47.000,61.000,244.000,12.0000,5900.0,298000.0,16.20,0.60
1717,322.000,95.0,1.6,150.0,6.0,1.000,0.2875,126.000,73.000,96.875,13.5750,24100.0,303000.0,116.00,2.80
1174,68.000,49.0,0.8,135.0,4.4,1.000,0.2625,58.000,35.000,67.000,12.5000,14500.0,413000.0,4.00,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,125.000,74.0,1.1,142.0,4.0,1.000,0.3000,46.000,33.000,83.000,16.0000,26000.0,305000.0,42.00,2.80
1247,251.000,82.0,1.3,132.0,4.5,0.900,0.3000,48.000,31.000,97.000,15.6000,10800.0,454000.0,10.00,0.50
271,87.000,118.0,2.0,141.0,4.4,0.800,0.2000,26.000,16.000,69.000,105.0000,5800.0,293000.0,0.00,0.30
474,95.000,81.0,1.6,140.0,4.4,1.000,0.3000,69.000,46.000,346.000,14.7000,26100.0,225000.0,131.00,2.70


In [85]:
# 决策树模型
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1824, n_features=17,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

decision_tree_model = RandomForestClassifier()
decision_tree_model.fit(X_train, y_train)

# 预测
y_pred = decision_tree_model.predict(X_test)

# 输出分类报告和混淆矩阵
print("分类报告:\n", classification_report(y_test, y_pred))
print("混淆矩阵:\n", confusion_matrix(y_test, y_pred))




分类报告:
               precision    recall  f1-score   support

           0       0.96      0.84      0.90        83
           1       0.96      0.99      0.97       282

    accuracy                           0.96       365
   macro avg       0.96      0.92      0.93       365
weighted avg       0.96      0.96      0.96       365

混淆矩阵:
 [[ 70  13]
 [  3 279]]


In [ ]:
import joblib

joblib.dump(decision_tree_model, 'dtc.pkl')